# Feature selection and intial predictions

In [1]:
%reload_ext autoreload
%autoreload 2
import os 
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import plotly 
import plotly.graph_objects as go
from datetime import timedelta

sys.path.insert(0, "./../../")

from src.preprocessing.feature_preparation import prepare_features_for_season_based_predictions
from src.preprocessing.DataPreparator import DataPreparator

In [2]:
data_p = DataPreparator()
all_games = data_p.get_games_df()

# home team wins in 60% of the cases
display(all_games["home_win"].mean())

Loading data from file C:\Users\Zan\Desktop\Faksic_TUM\2_semester\Praktikum_Data_mining\nba-data-mining\src\./../data/processed/load_data_games_arr_v2_zan.pkl ...
Data loaded!

Columns that will be used are of lengths: 
    - x_home_cols: 30, 
    - x_visit_cols length: 30


0.5966035271064664

Base line for predictions is 0.597 -> which is ~60%, that means if we predict that each game will be won by the home team we are correct in 60% of cases. Any improvements over 60% are actual benefits of our models.

In [3]:
x_train, y_train, x_test, y_test = data_p.prepare_data_splits("2012-2013")
display(x_train)
display(y_train)

array([[ 1.        ,  4.        , 48.        , ..., 28.        ,
         1.        , 13.73809524],
       [ 1.        ,  4.        , 48.        , ..., 27.        ,
         1.        ,  8.82352941],
       [ 1.        ,  4.        , 48.        , ..., 28.        ,
         1.        ,  9.97619048],
       ...,
       [ 1.        ,  4.        , 48.        , ..., 26.        ,
         1.        , 16.15151515],
       [ 1.        ,  4.        , 48.        , ..., 27.        ,
         0.        , 10.04255319],
       [ 1.        ,  4.        , 48.        , ..., 29.        ,
         0.        ,  9.90697674]])

array([1., 0., 1., ..., 0., 1., 0.])

# Feature selection
Simpler models wil be tested out of the box.

## Random forest

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score

r_forest = RandomForestRegressor(n_estimators=300, criterion='squared_error', random_state=42)
r_forest.fit(x_train, y_train)

# evaluate on train data set ->> overfitting but just to see the accuracy
print("Starting predictions ...")
y_pred_train = []
for row in x_train:
    y_pred_train.append(round(r_forest.predict([row])[0]))
    
print(f"Accuracy on the train set is: {accuracy_score(y_train, y_pred_train, normalize=True) * 100} %")

Starting predictions ...
Accuracy on the train set is: 100.0 %


Evaluation on the train set yield an accuracy of 100% which means model can overfit to the given data correctly!

In [5]:
print("Evaluating on the test data set!")

y_pred_test = []
for row in x_test:
    y_pred_test.append(round(r_forest.predict([row])[0]))
    
print(f"Accuracy on the train set is: {accuracy_score(y_test, y_pred_test, normalize=True) * 100} %")

Evaluating on the test data set!
Accuracy on the train set is: 81.95121951219512 %


Achieves around 83% prediction accuracy on the test data set when the split is of size 0.8, 0.1 and 0.1. And 82% if the data is lowered to 82%.